<p align="center">
  <a href="https://github.com/google/langextract">
    <img src="https://raw.githubusercontent.com/google/langextract/main/docs/_static/logo.svg"
         alt="LangExtract Logo" width="128" />
  </a>
</p>

<h1 align="center">LangExtract</h1>

<p align="center">
  <a href="https://pypi.org/project/langextract/">
    <img src="https://img.shields.io/pypi/v/langextract.svg" />
  </a>
  <a href="https://github.com/google/langextract">
    <img src="https://img.shields.io/github/stars/google/langextract.svg?style=social&label=Star" />
  </a>
  <img src="https://github.com/google/langextract/actions/workflows/ci.yaml/badge.svg" />
  <a href="https://doi.org/10.5281/zenodo.17015089">
    <img src="https://zenodo.org/badge/DOI/10.5281/zenodo.17015089.svg" />
  </a>
</p>

<p align="center">
  <img src="https://raw.githubusercontent.com/google/langextract/main/docs/_static/romeo_juliet_basic.gif"
       alt="Romeo and Juliet Basic Visualization" />
</p>


In [8]:
!pip install langextract


##  Google AI Studio

You can access it via **Google AI Studio** at the following link:  
🔗 https://aistudio.google.com/


In [ ]:
import os
os.environ["LANGEXTRACT_API_KEY"] = "YOUR_API_KEY"



In [7]:
import langextract as lx
import textwrap

# 1. Định nghĩa Prompt: Bạn muốn trích xuất cái gì?
prompt = textwrap.dedent("""\
    Trích xuất tên nhân vật, cảm xúc và mối quan hệ theo thứ tự xuất hiện.
    Sử dụng chính xác từ ngữ trong văn bản, không được viết lại (paraphrase).
""")

# 2. Tạo ví dụ mẫu (Few-shot example):
# Bước này rất quan trọng để dạy model cách trả về dữ liệu đúng ý bạn.
examples = [
    lx.data.ExampleData(
        text="ROMEO: Nhưng khoan! Ánh sáng nào lóe qua cửa sổ kia? Đó là phương Đông, và Juliet là mặt trời.",
        extractions=[
            lx.data.Extraction(
                extraction_class="character", # Loại dữ liệu
                extraction_text="ROMEO",      # Text chính xác trong văn bản
                attributes={"emotional_state": "kinh ngạc"} # Thuộc tính bổ sung
            ),
            lx.data.Extraction(
                extraction_class="relationship",
                extraction_text="Juliet là mặt trời",
                attributes={"type": "ẩn dụ"}
            ),
        ]
    )
]

# 3. Chạy trích xuất
input_text = "Nàng Juliet nhìn đắm đuối lên những vì sao, trái tim nàng đau nhói vì nhớ Romeo"

result = lx.extract(
    text_or_documents=input_text,
    prompt_description=prompt,
    examples=examples,
    model_id="gemini-2.5-pro", # Chọn model (nhanh và rẻ)
)

# 4. Xuất kết quả và tạo file HTML để xem
lx.io.save_annotated_documents([result], output_name="ket_qua.jsonl", output_dir=".")
html = lx.visualize("ket_qua.jsonl")

# Save the results to a JSONL file
lx.io.save_annotated_documents([result], output_name="ket_qua.jsonl", output_dir=".")

# Generate the visualization from the file
html_content = lx.visualize("ket_qua.jsonl")
with open("visualization.html", "w") as f:
    if hasattr(html_content, 'data'):
        f.write(html_content.data)  # For Jupyter/Colab
    else:
        f.write(html_content)


LangExtract: model=gemini-2.5-flash [00:00]
LangExtract: model=gemini-2.5-flash, current=79 chars, processed=0 chars:  [00:00]
LangExtract: model=gemini-2.5-flash, current=79 chars, processed=0 chars:  [00:23]

LangExtract: Saving to ket_qua_fix.jsonl: 1 docs [00:00, 784.28 docs/s]

✓ Saved 1 documents to ket_qua_fix.jsonl




LangExtract: Loading ket_qua_fix.jsonl: 100%|██████████| 1.81k/1.81k [00:00<00:00, 5.64MB/s]

✓ Loaded 1 documents from ket_qua_fix.jsonl
Đã sửa lỗi và lưu file thành công!


# Scaling to Longer Documents

In [13]:
import langextract as lx
import textwrap

# 1. Prompt & Example (Giữ nguyên hoặc dùng cái đơn giản)
prompt = textwrap.dedent("""\
    Extract main characters and their locations.
    Keep extractions verbatim from text.
""")

examples = [
    lx.data.ExampleData(
        text="Alice sat by the river bank.",
        extractions=[
            lx.data.Extraction(
                extraction_class="character",
                extraction_text="Alice",
                attributes={"location": "river bank"}
            )
        ]
    )
]

# 2. Chạy trích xuất từ URL
# Tài liệu ví dụ dùng Romeo & Juliet từ Project Gutenberg
print("Đang tải và xử lý văn bản từ URL...")
result = lx.extract(
    text_or_documents="https://www.gutenberg.org/files/1513/1513-0.txt", # Link văn bản
    prompt_description=prompt,
    examples=examples,
    model_id="gemini-3-flash-preview",
    # --- CÁC THAM SỐ NÂNG CAO ---
    extraction_passes=2,    # Quét 2 lần để tăng độ phủ (recall)
    max_workers=10,         # Xử lý song song 10 luồng cho nhanh
    max_char_buffer=2000    # Giới hạn kích thước ngữ cảnh (context)
)

# 3. Lưu kết quả
lx.io.save_annotated_documents([result], output_dir=".", output_name="url_test.jsonl")
print("Xong! Hãy visualize file url_test.jsonl")

Đang tải và xử lý văn bản từ URL...


LangExtract: Downloading https://www.gutenberg.org/files/1513/1513-0.txt: 100%|█| 145k/145k [00:00<0

✓ Downloaded 142,570 characters (25,976 words) from 1513-0.txt



LangExtract: model=gemini-3-flash-preview, current=19,807 chars, processed=0 chars:  [03:38]


KeyboardInterrupt: 